In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

# ResNet Implementation

In [6]:
# Function that returns a Conv Layer with 
# Kernel Size 3x3
# Stride Size of stride x stride (default = 1)
# Padding Size of 1x1
# Convolution takes in nc_in planes and gives an output of nc_out planes
def sameConv(nc_in, nc_out, stride=1):
    return nn.Conv2d(nc_in, nc_out, kernel_size=3, stride=stride, padding=1)

In [8]:
# The Basic Resnet Block with no BottleNeck used for smaller Resnet implementations
# Like ResNet18 and ResNet34
# The idea is that the n + 2 layer returns an activation of H(x) = g(F(x) + x)
# Instead of parameters that are randomly initialized we have an identity mapping
# This makes it much easier for the network to learn since it does not have to learn parameters
# from the scratch
class BasicResBlock(nn.Module):
    def __init__(self, nc_in, nc_out, stride=1):
        super(BasicResBlock, self).__init__()
        self.conv1 = sameConv(nc_in, nc_out)
        self.bn1 = nn.BatchNorm2d(nc_out)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = sameConv(nc_out, nc_out)
        self.bn2 = nn.BatchNorm2d(nc_out)
        self.stride = stride
        
    def forward(self, x):
        residual = x
        
        activation = self.conv1(x)
        activation = self.bn1(x)
        activation = self.relu(x)
        activation = self.conv2(x)
        activation = self.bn2(x)
        activation += residual
        
        activation = self.relu(activation)
        
        return activation

In [5]:
class ResNet(nn.Module):
    
    def __init__(self, block, layers, num_classes=1000):
        self.inplanes = 64
        super(ResNet, self).__init__()
        
        #7x7 Convolution with Stride 2 and padding for 3
        #Followed by a batch normalization and max pooling
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # Make the resnet layers
        # The first layer is layers[0] number of ResidualBlocks with plane size of 64
        # The second layer is layers[1] number of ResidualBlocks with plane size of 128
        # The third layer is layers[2] number of ResidualBlocks with plane size of 256
        # The fourth layer is layers[3] number of ResidualBlocks with plane size of 512
        # The basic rule of ResNet is that everytime we decrease the the input size with
        # with a convolution we double the number of channels/planes
        self.layer1 = _make_resnet_layer(block, 64, layers[0])
        self.layer2 = _make_resnet_layer(block, 128, layers[1])
        self.layer3 = _make_resnet_layer(block, 256, layers[2])
        self.layer4 = _make_resnet_layer(block, 512, layers[3])
        
        #Average Pooling and then fully connected layer
        self.avgpool = AvgPool2d(7, stride=1)
        
        # A fully connected layer with 512 * block.expansion number of inputs
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        
        #Need to figure out what this is doing
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n - m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
    
    #Function that makes a layer of ResNet Blocks with 
    def _make_resnet_layer(self, block, nc_out, blocks, stride=1):
        layers = []
        
        layers.append(block(self.inplanes, planes, stride, None))
        self.inplanes = planes * block.explansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, nc_out))
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.ReLU(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
    
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        return x